# Lab 5: Land Surface Temperature using CORINE-based Emissivity

## 🎯 Objectives
In this exercise, you will:
- Select a cloud-free Landsat 8 images from 2013 and 2023 (or different if you're sure that you'll spot a difference in land cover)
- Calculate Brightness Temperature (TB) from Band 10.
- Load CORINE Land Cover data and assign emissivity values to each land cover class.
- Use the Planck-based formula to calculate Land Surface Temperature (LST).
- Visualize and interpret the results.

## Step 1: Define Area of Interest (AOI)
- Use coordinates around Reduta street in Kraków.
- Create a polygon or rectangle using `ee.Geometry.Polygon`.

In [1]:
# Define AOI here

import ee

ee.Authenticate()
ee.Initialize(project='ee-martynadurda')

aoi = ee.Geometry.Rectangle([-3.70239, 40.80757, -3.52386, 40.04233])  # cords of madrid bb

print(aoi.getInfo())

{'type': 'Polygon', 'coordinates': [[[-3.70239, 40.04233], [-3.52386, 40.04233], [-3.52386, 40.80757], [-3.70239, 40.80757], [-3.70239, 40.04233]]]}


## Step 2: Load Landsat 8 imagery for the dates you've picked
- Filter for low cloud cover (< 20%)
- Select Band 10 and convert to TB using: `TB = ST_B10 * 0.00341802 + 149.0`

In [2]:
# Load image and calculate TB
import geemap 
start_2013 = ee.Date('2013-08-01') # dzien wczesniej bo na 08 nie ma zdj
end_2013 = ee.Date('2013-08-15')
start_2024 = ee.Date('2023-08-06') # zakres 26-31 bo i tak wszystko >27 stopni
end_2024 = ee.Date('2023-08-16')

image_2013 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterBounds(aoi).filterDate(start_2013, end_2013).filterMetadata('CLOUD_COVER', 'less_than', 20).first()
image_2024 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').filterBounds(aoi).filterDate(start_2024, end_2024).filterMetadata('CLOUD_COVER', 'less_than', 20).first()
image_2013 = image_2013.clip(aoi)
image_2024 = image_2024.clip(aoi)

print(image_2013.getInfo())
print(image_2024.getInfo())

vis_params = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 5000,
    'max': 15000,
    'gamma': 1.3
}
center = aoi.centroid(10).coordinates().reverse().getInfo()

Map = geemap.Map(center=center, zoom=15)

Map.addLayer(image_2013, vis_params, 'Landsat 2013-08-08')
Map.addLayer(image_2024, vis_params, 'Landsat 2024-08-25')

Map

{'type': 'Image', 'bands': [{'id': 'SR_B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [526, 2836], 'origin': [4169, 2182], 'crs': 'EPSG:32630', 'crs_transform': [30, 0, 314985, 0, -30, 4583115]}, {'id': 'SR_B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [526, 2836], 'origin': [4169, 2182], 'crs': 'EPSG:32630', 'crs_transform': [30, 0, 314985, 0, -30, 4583115]}, {'id': 'SR_B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [526, 2836], 'origin': [4169, 2182], 'crs': 'EPSG:32630', 'crs_transform': [30, 0, 314985, 0, -30, 4583115]}, {'id': 'SR_B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [526, 2836], 'origin': [4169, 2182], 'crs': 'EPSG:32630', 'crs_transform': [30, 0, 314985, 0, -30, 4583115]}, {'id': 'SR_B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 6553

Map(center=[40.42425881714212, -3.6131250000012645], controls=(WidgetControl(options=['position', 'transparent…

In [9]:
tb_2013 = image_2013.select('ST_B10').multiply(0.00341802).add(149.0)
tb_2024 = image_2024.select('ST_B10').multiply(0.00341802).add(149.0)




## Step 3: Load CORINE Land Cover data
- Use dataset `COPERNICUS/CORINE/V20/100m/2018`
- Clip it to your AOI

In [5]:
emissivity_dict = {
    111: 0.92,  # Continuous urban fabric
    112: 0.92,  # Discontinuous urban fabric
    121: 0.91,  # Industrial or commercial units
    211: 0.96,  # Non-irrigated arable land
    311: 0.98,  # Forests
    412: 0.97,  # Peat bogs
    324: 0.96,  # Transitional woodland-shrub
    231: 0.97   # Pastures
}

# Load CORINE
corine = ee.Image('COPERNICUS/CORINE/V20/100m/2018').clip(aoi)

# Assign emissivity values to CORINE classes
emissivity = corine.remap(
    list(emissivity_dict.keys()),
    list(emissivity_dict.values()),
    0.95  # Default emissivity value
)

# Add CORINE and emissivity layers to the map
Map.addLayer(corine, {'min': 111, 'max': 231, 'palette': ['blue', 'green', 'yellow', 'red']}, 'CORINE Land Cover')
Map.addLayer(emissivity, {'min': 0.9, 'max': 1.0, 'palette': ['blue', 'green', 'yellow', 'red']}, 'Emissivity')
Map

Map(bottom=396297.0, center=[40.255162653545, -3.7944030761718754], controls=(WidgetControl(options=['position…

## Step 4: Assign emissivity to CORINE classes
- Use a dictionary for classes
- Use `remap()` and optionally a default value

In [ ]:
# Create emissivity image
#coppied above 
emissivity_dict = {
    111: 0.92,  # Continuous urban fabric
    112: 0.92,  # Discontinuous urban fabric
    121: 0.91,  # Industrial or commercial units
    211: 0.96,  # Non-irrigated arable land
    311: 0.98,  # Forests
    412: 0.97,  # Peat bogs
    324: 0.96,  # Transitional woodland-shrub
    231: 0.97   # Pastures
}

## Step 5: Calculate LST using the formula:
$$
LST = \frac{T_B}{1 + \left( \frac{\lambda \cdot T_B}{c_2} \right) \cdot \ln(\varepsilon)}
$$
- λ = 10.8 µm
- c₂ = 14388 µm·K

In [15]:
# Calculate LST using Earth Engine methods
lambda_ = 10.8e-6  # Wavelength in meters (10.8 µm)
c2 = 14388  # Second radiation constant in µm·K

# Assuming TB (brightness temperature) and emissivity are ee.Image objects
TB = tb_2024  # Brightness temperature in Kelvin (ee.Image)
epsilon = emissivity  # Emissivity (ee.Image)

# Calculate LST
LST = TB.divide(
    ee.Image(1).add(
        ee.Image(lambda_).multiply(TB).divide(c2).multiply(epsilon.log())
    )
)


## Step 6: Visualize the LST result
- Use palette: `['blue', 'yellow', 'red']`
- Suggested range: `min=290`, `max=325`

In [17]:
# Visualize LST
# Visualize LST
Map.addLayer(LST, {
    'min': 290,
    'max': 325,
    'palette': ['blue', 'yellow', 'red']
}, 'Land Surface Temperature')

# Display the map
Map

Map(bottom=99347.0, center=[40.20509926855807, -3.607674879957568], controls=(WidgetControl(options=['position…

## Step 7: (Optional) Analyze statistics by land cover class

In [18]:
# Optionally compute zonal statistics
# Step 7: Analyze statistics by land cover class

# Define a region of interest (ROI) if not already defined
roi = aoi  # Replace with your actual region of interest

# Group LST by land cover classes
land_cover_classes = corine  # CORINE land cover image
stats = LST.addBands(land_cover_classes).reduceRegion(
    reducer=ee.Reducer.mean().combine(
        reducer2=ee.Reducer.median(),
        sharedInputs=True
    ).combine(
        reducer2=ee.Reducer.stdDev(),
        sharedInputs=True
    ),
    geometry=roi,
    scale=100,  # Adjust scale to match your data resolution
    maxPixels=1e13
)

# Print the statistics
print('LST Statistics by Land Cover Class:', stats.getInfo())

LST Statistics by Land Cover Class: {'ST_B10_mean': 321.13450397685006, 'ST_B10_median': 321.873514792864, 'ST_B10_stdDev': 4.322308044850877, 'landcover_mean': 217.34364877933746, 'landcover_median': 211.0000000000002, 'landcover_stdDev': 80.11511996579573}


## Step 8: (Optional - Easter Egg :)) Generate your own Land Cover Classification using TerraTorch and foundation models*

Based on the example/tutorial: https://aiforgood.itu.int/event/workshop-earth-observation-foundation-models-with-prithvi-eo-2-0-and-terratorch/

*to earn 5.0 grade that will make a great impact on your final grade